In [7]:
# concurrent.futures map
# concurrent.futures wait, as_completed

# 작업의 양이 작으면 wait으로 처리, 다 기다렸다가 한번에 취합해서 DB 작업인 경우 wait
# 하나라도 끝나는대로 작업을 해야하는 Real-Time성의 작업은 as_completed

import os
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed

WORK_LIST = [100000, 1000000, 10000000, 100000000]

# 동시성 합계 계산 메인 함수
# 누적 합계 함수(제네레이터)
def sum_generator(n):
    return sum(n for n in range(1,n+1))

def main():
    # Worker Count
    worker = min(10, len(WORK_LIST))
    
    # 시작 시간
    start_tm = time.time()
    # Futures
    futures_list = []
    # 결과 건수
    # ProcessPoolExecutor: 프로세스 사용
    # ThreadPoolExecutor: 현재 프로세스의 쓰레드 사용
    with ThreadPoolExecutor() as executor:
        for work in WORK_LIST:
            # future 반환
            future = executor.submit(sum_generator, work)
            # 스케쥴링
            futures_list.append(future)
            # 스케쥴링 확인
            print('Schduled for {} : {}'.format(work, future))
            print()
            
        result = wait(futures_list, timeout=5) # 7초가 지난 뒤 끝나지 못한 일은 실패로 간주
        # 성공
        print('Completed Tasks: ' + str(result.done))
        # 실패
        print('Pending ones after wairing for 5 seconds: ' + str(result.not_done))
        # 결과 값 출력
        print([future.result() for future in result.done])
        
    # 종료 시간
    end_tm = time.time() - start_tm
    # 출력 포멧
    msg = '\n Time : {:.2f}s'
    # 최종 결과 출력
    print(msg.format(end_tm))
    
# 실행
if __name__ == '__main__':
    main()

Schduled for 100000 : <Future at 0x17becd53b50 state=finished returned int>

Schduled for 1000000 : <Future at 0x17becd553a0 state=pending>

Schduled for 10000000 : <Future at 0x17becd554f0 state=running>

Schduled for 100000000 : <Future at 0x17becdaf730 state=pending>

Completed Tasks: {<Future at 0x17becd553a0 state=finished returned int>, <Future at 0x17becd53b50 state=finished returned int>, <Future at 0x17becd554f0 state=finished returned int>}
Pending ones after wairing for 5 seconds: {<Future at 0x17becdaf730 state=running>}
[500000500000, 5000050000, 50000005000000]

 Time : 6.99s


In [11]:

# 동시성 합계 계산 메인 함수
# 누적 합계 함수(제네레이터)
def sum_generator2(n):
    return sum(n for n in range(1,n+1))

def main2():
    # Worker Count
    worker = min(10, len(WORK_LIST))
    
    # 시작 시간
    start_tm = time.time()
    # Futures
    futures_list = []
    # 결과 건수
    # ProcessPoolExecutor: 프로세스 사용
    # ThreadPoolExecutor: 현재 프로세스의 쓰레드 사용
    with ThreadPoolExecutor() as executor:
        for work in WORK_LIST:
            # future 반환
            future = executor.submit(sum_generator2, work)
            # 스케쥴링
            futures_list.append(future)
            # 스케쥴링 확인
            print('Schduled for {} : {}'.format(work, future))
            print()
            
        for future in as_completed(futures_list):
            result = future.result()
            done = future.done()
            cancelled = future.cancelled
            
            # future 결과 확인
            print('Future Result: {}, Done: {}'.format(result, done))
            print('Future Cancelled: {}'.format(cancelled))
    
    # 종료 시간
    end_tm = time.time() - start_tm
    # 출력 포멧
    msg = '\n Time : {:.2f}s'
    # 최종 결과 출력
    print(msg.format(end_tm))
    
# 실행
if __name__ == '__main__':
    main2()

Schduled for 100000 : <Future at 0x17becdacf40 state=finished returned int>

Schduled for 1000000 : <Future at 0x17becdacd90 state=pending>

Schduled for 10000000 : <Future at 0x17becd24760 state=running>

Schduled for 100000000 : <Future at 0x17becd247f0 state=running>

Future Result: 5000050000, Done: True
Future Cancelled: <bound method Future.cancelled of <Future at 0x17becdacf40 state=finished returned int>>
Future Result: 500000500000, Done: True
Future Cancelled: <bound method Future.cancelled of <Future at 0x17becdacd90 state=finished returned int>>
Future Result: 50000005000000, Done: True
Future Cancelled: <bound method Future.cancelled of <Future at 0x17becd24760 state=finished returned int>>
Future Result: 5000000050000000, Done: True
Future Cancelled: <bound method Future.cancelled of <Future at 0x17becd247f0 state=finished returned int>>

 Time : 7.22s
